In [1]:
# Running different evaluations for long-doc-summ

In [2]:
orig_path = "booksumm-sample2/what-maisie-knew"

In [3]:
import json

with open(orig_path+"/metadata.json") as f:
  book_metadata = json.load(f)

In [4]:
book_metadata

{'title': 'What Maisie Knew',
 'author': 'Henry James',
 'total-chapters': 31,
 'average-chapter-len': 4256.935483870968,
 'dir': '7118-chapters'}

In [5]:
import os
import json
import pandas as pd

# Define directory paths
dir_path1 = "booksumm-sample2/what-maisie-knew"
dir_path2 = "gpt4_summaries/what-maisie-knew"

# Define the data structure
data = {
    "subdir": [],
    "metadata": [],
    "content": [],
    "summaries": [],
    "new_summary": []
}

# Loop over subdirectories
for subdir in os.listdir(dir_path1):
    subdir_path1 = os.path.join(dir_path1, subdir)

    # If the path is a directory, process it
    if os.path.isdir(subdir_path1):
        data["subdir"].append(subdir)

        # Process metadata.json
        with open(os.path.join(subdir_path1, "metadata.json"), 'r') as f:
            data["metadata"].append(json.load(f))

        # Process content.txt
        with open(os.path.join(subdir_path1, "content.txt"), 'r') as f:
            data["content"].append(f.read())

        # Process summaries
        summaries = []
        for file in os.listdir(subdir_path1):
            if file.endswith("summary.txt"):
                with open(os.path.join(subdir_path1, file), 'r') as f:
                    content = f.read()
                    summaries.append(content)
        data["summaries"].append(summaries)

        # Process new directory
        subdir_path2 = os.path.join(dir_path2, subdir)
        if os.path.isdir(subdir_path2):
            with open(os.path.join(subdir_path2, "summary.txt"), 'r') as f:
                data["new_summary"].append(f.read())
        else:
            data["new_summary"].append(None)

# Create a DataFrame
df = pd.DataFrame(data)
df = df.set_index('subdir')  # Set index to subdirectory names


json.loads(df['summaries'][0][0])["summary"]


'Maisie has to go back to her father and Miss Overmore . This means that she has to say goodbye to Mrs. Wix for the first time, and this is super, super sad! Maisie asks her father if he liked Miss Overmore "just the same" while she was at her mother\'s . This embarrasses Miss Overmore, even though Beale Farange makes no secret of having been with Miss Overmore the whole time. This is just above Maisie\'s head. She, too, feels embarrassed and wonders why Miss Overmore has been awkward. She also tries to figure out the nature of her father\'s relationship to Miss Overmore. Maisie takes to treating her doll, Lisette, the way she herself is treated by her mother and Miss Overmore. She keeps secrets from her doll and even reprimands her for asking indiscreet questions.'

In [6]:
df

,metadata,content,summaries,new_summary
subdir,,,,
05,"{'book-id': '7118-chapters', 'chapter-id': '05...",\n\nThe second parting from Miss Overmore had ...,"[{""name"": ""Chapter V"", ""url"": ""https://web.arc...",This chapter revolves around a young girl name...
21,"{'book-id': '7118-chapters', 'chapter-id': '21...",\n\nA good deal of the rest of Ida's visit was...,"[{""name"": ""Chapter XXI"", ""url"": ""https://web.a...","In this chapter, Maisie and her mother, Ida, s..."
30,"{'book-id': '7118-chapters', 'chapter-id': '30...",\n\nAfter they were seated there it was differ...,"[{""name"": ""Chapter XXX"", ""url"": ""https://web.a...","In this chapter, Sir Claude and Maisie have a ..."
23,"{'book-id': '7118-chapters', 'chapter-id': '23...",\n\nSir Claude was stationed at the window; he...,"[{""name"": ""Chapter XXIII"", ""url"": ""https://web...","In this chapter, Mrs. Wix, Sir Claude, and Mai..."
14,"{'book-id': '7118-chapters', 'chapter-id': '14...",\n\nMrs Beale fairly swooped upon her and the ...,"[{""name"": ""Chapter XIV"", ""url"": ""https://web.a...","In this chapter, Maisie is reunited with her s..."


In [7]:
def process_summary(summary_list):
  new_list = []
  for summary in summary_list:
    out = json.loads(summary)["summary"]
    new_list.append(out)
  return new_list


In [8]:
df["summaries"] = df['summaries'].apply(process_summary)

In [9]:
df

,metadata,content,summaries,new_summary
subdir,,,,
05,"{'book-id': '7118-chapters', 'chapter-id': '05...",\n\nThe second parting from Miss Overmore had ...,[Maisie has to go back to her father and Miss ...,This chapter revolves around a young girl name...
21,"{'book-id': '7118-chapters', 'chapter-id': '21...",\n\nA good deal of the rest of Ida's visit was...,[Ida and Maisie have a heart-to-heart. Ida is ...,"In this chapter, Maisie and her mother, Ida, s..."
30,"{'book-id': '7118-chapters', 'chapter-id': '30...",\n\nAfter they were seated there it was differ...,[Maisie's cafe trip with Sir Claude turns into...,"In this chapter, Sir Claude and Maisie have a ..."
23,"{'book-id': '7118-chapters', 'chapter-id': '23...",\n\nSir Claude was stationed at the window; he...,[Mrs. Wix says that she has seen Maisie's moth...,"In this chapter, Mrs. Wix, Sir Claude, and Mai..."
14,"{'book-id': '7118-chapters', 'chapter-id': '14...",\n\nMrs Beale fairly swooped upon her and the ...,[Mrs. Beale lets it slip that she and Sir Clau...,"In this chapter, Maisie is reunited with her s..."


In [10]:
%pip install rouge_score
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [58]:
import evaluate

rouge_score = evaluate.load("rouge")

In [67]:
meteor_score = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mbondarenko/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mbondarenko/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mbondarenko/nltk_data...


In [11]:
scores = meteor_score.compute(
    predictions=[df["new_summary"][0]], references=[df["summaries"][0]]
)
scores

NameError: name 'meteor_score' is not defined

In [66]:
scores = rouge_score.compute(
    predictions=[df["new_summary"][0]], references=[df["summaries"][0]]
)
scores

ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ["This chapter revolves around a young girl named Maisie, who is caught in the middle of her parents' complicated relationships and their various companions. The chapter begins with Maisie parting from Mrs. Wix, a scene described as being as intense and painful as a visit to the dentist. Maisie is then collected by her father, Beale Farange, and her former governess, Miss Overmore, who is now her father's companion. The child recalls previous separations and is aware of the tension and conflicting emotions involved in these transitions.\n\nMaisie's father and Miss Overmore engage in playful banter, which confuses and slightly embarrasses Maisie, as she does not fully understand the adult implications. She asks questions about their relationship and where Miss Overmore has

In [69]:
rouge = evaluate.load('rouge')
predictions = ["hello there", "general kenobi"]
references = [["hello", "there"], ["general kenobi", "general yoda"]]
results = rouge.compute(predictions=predictions,references=references)